In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/inlpa33/ANLP-2/train.csv
/kaggle/input/inlpa33/ANLP-2/test.csv


In [2]:
train_files=pd.read_csv('/kaggle/input/inlpa33/ANLP-2/train.csv')
test_files=pd.read_csv('/kaggle/input/inlpa33/ANLP-2/test.csv')

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize 
import re

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import io
import torch
from torch import nn,tensor
from torch.utils.data import Dataset,DataLoader
from torchtext.vocab import build_vocab_from_iterator
from tqdm import tqdm
import re
import random

In [5]:
stop_words=set(stopwords.words('english'))

In [6]:
train_files.head(1)

,Class Index,Description
0,3,"Reuters - Short-sellers, Wall Street's dwindli..."


**Reading Dataset's Description column and tokenizing sentences**

In [7]:
def tokenized_removed_sentence(df):
    tokenized_removed_sentence=[]
    for i in df['Description']:
        filtered_sentence=re.sub('\W+',' ', i)
        filtered_sentence = [w for w in word_tokenize(filtered_sentence)]
        tokenized_removed_sentence.append(filtered_sentence)
    df_one=pd.DataFrame.from_dict({'Description':tokenized_removed_sentence})
    df.Description=df_one.Description

In [8]:
tokenized_removed_sentence(train_files)

In [9]:
tokenized_removed_sentence(test_files)

In [10]:
import gc
gc.collect()

0

**Creating Co-occurance matrix for the SVD**

In [11]:
train_files_coocur = train_files.sample(frac = 1)

In [12]:
#bigram_for_cooc=[]
unigram_for_cooc=[]
for i in train_files_coocur.Description:
   # bigram_for_cooc.extend(list(bigrams(i)))
    unigram_for_cooc.extend(i)
from collections import Counter
#bigram_counter=Counter(bigram_for_cooc)
unigram_counter=Counter(unigram_for_cooc)
vocab_list_for_coocur_matrix=list(unigram_counter.keys())

In [13]:
vocab_list_for_coocur_matrix=['<PAD>']+['<UNK>']+vocab_list_for_coocur_matrix

**Creating entries for Co-occurMatrix**

In [14]:
word_to_ind={}
ind_to_word={}
for ind,word in enumerate(vocab_list_for_coocur_matrix):
    word_to_ind[word]=ind
    ind_to_word[ind]=word

In [15]:
word_to_ind.get('software')

439

In [16]:
len(vocab_list_for_coocur_matrix)

73635

**creating SkipGramModule where adding in_emb and context_emb unifrom distr. weights and using multinomial sampling for negative samples**

In [18]:
class SkipGramModule(nn.Module):
    def __init__(self,n_vocab,n_embed,noise_dist=None):
        super().__init__()
        self.n_vocab=n_vocab
        self.n_embed=n_embed
        self.noise_dist=noise_dist
        self.in_emb=nn.Embedding(n_vocab,n_embed)
        self.context_emb=nn.Embedding(n_vocab,n_embed)
        self.in_emb.weight.data.uniform_(-1,1)
        self.context_emb.weight.data.uniform_(-1,1)
    def forward_inp(self,input_words):
        input_emb=self.in_emb(input_words)
        return input_emb
    def forward_context_words(self,context_words):
        context_emb=self.context_emb(context_words)
        return context_emb
    def forward_negative(self,batch_size,n_samples):
        if self.noise_dist is None:
            noise_dist=torch.ones(self.n_vocab)
        else:
            noise_dist=self.noise_dist
        noise_words=torch.multinomial(noise_dist,batch_size*n_samples,replacement=True)
        noise_words=noise_words.to(device)
        noise_emb=self.context_emb(noise_words).view(batch_size,n_samples,self.n_embed)
        return noise_emb

In [19]:
class SGNS(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,input_emb,context_emb,noise_emb):
        batch_size,embed_size=input_emb.shape
        
        input_emb=input_emb.view(batch_size,embed_size,1)
        context_emb=context_emb.view(batch_size,1,embed_size)
        context_loss=torch.bmm(context_emb,input_emb).sigmoid().log()
        context_loss=context_loss.squeeze()
        
        noise_loss=torch.bmm(noise_emb.neg(),input_emb).sigmoid().log()
        noise_loss=noise_loss.squeeze().sum(1)
        return -(context_loss+noise_loss).mean()

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
word_freqs=np.array(sorted(unigram_counter.values(),reverse=True))
unigram_dist=word_freqs/word_freqs.sum()

In [24]:
noise_dist=torch.from_numpy(unigram_dist**(0.75)/np.sum(unigram_dist**(0.75)))

In [26]:
emb_dim=300
model=SkipGramModule(len(vocab_list_for_coocur_matrix),emb_dim,noise_dist).to(device)

In [ ]:
sent_corpus=[]
for i in train_files.Description:
    sent_corpus.extend(i)

In [ ]:
len(sent_corpus)

**for 1 positive using 5 as skip gram context length**

In [ ]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = words[start:idx] + words[idx+1:stop+1]
    
    return list(target_words)

def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    n_batches = len(words)//batch_size
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [ ]:
def create_batches(words,batch_size,window_size=5):
    n_batches=len(words)//batch_size
    #print(n_batches)
    words=words[:n_batches*batch_size]
    for indices in range(0,len(words),batch_size):
        data,label=[],[]
        batch=words[indices:indices+batch_size]
        for sample in range(len(batch)):
            batch_data=batch[sample]
            random_sampling=np.random.randint(1,window_size-1)
            start_id=sample-random_sampling if (sample-random_sampling)>0 else 0
            stop_id=sample+random_sampling
            context_words=words[start_id:sample]+words[sample+1:stop_id+1]
            batch_context=list(context_words)
            data.extend([batch_data]*len(batch_context))
            label.extend(batch_context)
        yield data,label

In [21]:
loss_fn=SGNS()


In [27]:
optimizer=torch.optim.Adam(model.parameters(),0.001)

**Training and Reporting the next set of batches**

In [ ]:
epoch_loss=[]
for e in range(50):
    batch_loss=0
    for data,label in tqdm(create_batches(sent_corpus,8192*2,5),desc='Generate batches and Loss'):
        optimizer.zero_grad()
        data=[word_to_ind.get(i,1) for i in data]
        label=[word_to_ind.get(i,1) for i in label]
        inp,context=torch.LongTensor(data),torch.LongTensor(label)
        inp,context=inp.to(device),context.to(device)
        inp_emb=model.forward_inp(inp)
        context_emb=model.forward_context_words(context)
        negative_samples=model.forward_negative(inp.shape[0],3)
        loss=loss_fn(inp_emb,context_emb,negative_samples)
        #print(loss.item())
        batch_loss+=loss.item()
        loss.backward()
        optimizer.step()
    print(batch_loss)
    epoch_loss.append(batch_loss)

In [ ]:
epoch_loss_context_window5=epoch_loss

In [ ]:
torch.save(model,'./model_context_win_5_emb_300')

In [ ]:
model2=SkipGramModule(len(vocab_list_for_coocur_matrix),emb_dim,noise_dist).to(device)

In [ ]:
def train_model(model,context_window,epochs):
    epoch_loss=[]
    optimizer=torch.optim.Adam(model.parameters(),0.005)
    for e in range(epochs):
        batch_loss=0
        for data,label in tqdm(create_batches(sent_corpus,8192*2,context_window),desc='Generate batches and Loss'):
            optimizer.zero_grad()
            data=[word_to_ind.get(i,1) for i in data]
            label=[word_to_ind.get(i,1) for i in label]
            inp,context=torch.LongTensor(data),torch.LongTensor(label)
            inp,context=inp.to(device),context.to(device)
            inp_emb=model.forward_inp(inp)
            context_emb=model.forward_context_words(context)
            negative_samples=model.forward_negative(inp.shape[0],3)
            loss=loss_fn(inp_emb,context_emb,negative_samples)
            #print(loss.item())
            batch_loss+=loss.item()
            loss.backward()
            optimizer.step()
        print(batch_loss)
        epoch_loss.append(batch_loss)    
    return epoch_loss

In [ ]:
epoch_loss_context_win_7=train_model(model2,7,25)
torch.save(model2,'./model_context_win_7_emb_300')

In [ ]:
model3=SkipGramModule(len(vocab_list_for_coocur_matrix),emb_dim,noise_dist).to(device)
epoch_loss_context_win_9=train_model(model3,9,15)
torch.save(model3,'./model_context_win_9_emb_300')

In [ ]:
word_to_ind.get('<PAD>')

In [28]:
train_files_coocur['Class Index']=train_files_coocur['Class Index'].apply(lambda x:x-1)
train_files_coocur['Class Index'].unique()

array([3, 2, 0, 1])

In [29]:
##max seq len will be 55 rest will be truncated

class Datafetch(Dataset):
    def __init__(self,type_of_data):
        data=[]
        labels=[]
        for y,x in tqdm(type_of_data,desc='Generate Data and Labels'):
            indices=[word_to_ind.get(word,1) for word in x['Description']]
            #indices=[0 if word==-1 else word for word in indices]
            if len(indices)<55:
                indices=indices+(55-len(indices))*[0]
            elif len(indices)>55:
                indices=indices[:55]
            data.append(indices)
            labels.append(x['Class Index'])
        self.data=torch.tensor(data)
        self.labels=torch.tensor(labels)
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        return self.data[idx],self.labels[idx]

In [30]:
train_files['Class Index']=train_files['Class Index'].apply(lambda x:x-1)
train_files['Class Index'].unique()

array([2, 3, 1, 0])

In [31]:
train_dataset=Datafetch(train_files.iterrows())

Generate Data and Labels: 120000it [00:08, 13427.68it/s]


In [32]:
train_dataloader=DataLoader(train_dataset,batch_size=2048)

In [33]:
test_files['Class Index']=test_files['Class Index'].apply(lambda x:x-1)

In [34]:
test_dataset=Datafetch(test_files.iterrows())
test_dataloader=DataLoader(test_dataset,batch_size=2048)

Generate Data and Labels: 7600it [00:00, 14093.17it/s]


In [35]:
from torchmetrics import Accuracy, Precision, Recall, F1Score
num_classes = 4
accuracy = Accuracy(task='multiclass',num_classes=num_classes).to(device)
precision = Precision(task='multiclass', average='macro', num_classes=num_classes).to(device)
recall = Recall(task='multiclass', average='macro', num_classes=num_classes).to(device)
f1 = F1Score(task='multiclass', average='macro', num_classes=num_classes).to(device)

In [36]:
class LSTM1(torch.nn.Module):
    def __init__(self,embedding,embedding_dim,hidden_dim,output_labels_size):
        super(LSTM1,self).__init__()
        self.embedding=embedding
        #self.embedding.weight.requires_grad=False
        self.lstm=nn.LSTM(embedding_dim,hidden_dim,num_layers=2,bidirectional=True,batch_first=True)
        #self.linear1=nn.Linear(hidden_dim*2,out_features=384)
        self.linear2=nn.Linear(hidden_dim*2,output_labels_size)
        #self.lrelu=nn.LeakyReLU()
    def forward(self,id_data):
        x=self.embedding(id_data)
        x,(o_t,y_t)=self.lstm(x)
        #x=self.lrelu(self.linear1(x))
        x=self.linear2(x[:,-1,:])
        return x

*using context length as 7 and 9 for the rest of the purposes**

In [38]:
model=torch.load('/kaggle/working/model_context_win_5_emb_300')
model2=torch.load('/kaggle/working/model_context_win_7_emb_300')
model3=torch.load('/kaggle/working/model_context_win_9_emb_300')

In [39]:
context5=model.in_emb
context7=model2.in_emb
context9=model3.in_emb

In [40]:
lstm_5=LSTM1(context5,300,512,4).to(device)

In [54]:
def train_epoch(lstm,dataloader1,device):
    loss_train=0
    accuracy_train=0
    precision_train=0
    recall_train=0
    f1_train=0
    loss_fn=torch.nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(lstm.parameters(),0.01)
    lstm.train()
    for batch_dl in tqdm(dataloader1,desc="train"):
        d,l=batch_dl
        optimizer.zero_grad()
        out=lstm(d.to(device))
        bloss=loss_fn(out,l.to(device))
        bloss.backward()
        optimizer.step()
        loss_train+=bloss.item()
        preds=torch.argmax(out,dim=1)
        accuracy_train+=accuracy(preds,l.to(device))
        precision_train+=precision(preds,l.to(device))
        recall_train+=recall(preds,l.to(device))
        f1_train+=f1(preds,l.to(device))
    return loss_train,accuracy_train/len(dataloader1),precision_train/len(dataloader1),recall_train/len(dataloader1),f1_train/len(dataloader1)

In [55]:
def train(lstm,device_1):
    tr_loss=[]
    acc_train=[]
    precision_train=[]
    recall_train=[]
    f1_train=[]
    for i in range(15):
        tr,acc,pr,re,f1=train_epoch(lstm,train_dataloader,device_1)
        print(tr)
        tr_loss.append(tr)
        acc_train.append(acc)
        precision_train.append(pr)
        recall_train.append(re)
        f1_train.append(f1)
    return tr_loss,acc_train,precision_train,recall_train,f1_train

In [43]:
tr_l,acc_train,precison_train,recall_train,f1_train=train(lstm_5,device)

train: 100%|██████████| 59/59 [00:57<00:00,  1.02it/s]


101.15996837615967


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


88.57003545761108


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


...

5.909060977399349


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


5.314709659665823


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]

5.569721128791571


In [44]:
torch.save(lstm_5,'./lstm5')

In [46]:
print(acc_train,precison_train,recall_train,f1_train)

[tensor(0.2463, device='cuda:0'), tensor(0.2586, device='cuda:0'), tensor(0.3750, device='cuda:0'), tensor(0.6647, device='cuda:0'), tensor(0.8219, device='cuda:0'), tensor(0.8916, device='cuda:0'), tensor(0.9161, device='cuda:0'), tensor(0.9337, device='cuda:0'), tensor(0.9448, device='cuda:0'), tensor(0.9498, device='cuda:0'), tensor(0.9578, device='cuda:0'), tensor(0.9647, device='cuda:0'), tensor(0.9691, device='cuda:0'), tensor(0.9725, device='cuda:0'), tensor(0.9715, device='cuda:0')] [tensor(0.1787, device='cuda:0'), tensor(0.2352, device='cuda:0'), tensor(0.4308, device='cuda:0'), tensor(0.6820, device='cuda:0'), tensor(0.8432, device='cuda:0'), tensor(0.9005, device='cuda:0'), tensor(0.9196, device='cuda:0'), tensor(0.9340, device='cuda:0'), tensor(0.9450, device='cuda:0'), tensor(0.9501, device='cuda:0'), tensor(0.9576, device='cuda:0'), tensor(0.9648, device='cuda:0'), tensor(0.9693, device='cuda:0'), tensor(0.9728, device='cuda:0'), tensor(0.9720, device='cuda:0')] [tensor(

In [57]:
def eval_func(lstm):
    lstm.eval()
    loss_eval=0
    accuracy_eval=0
    precision_eval=0
    recall_eval=0
    f1_eval=0
    preds_p=[]
    target_p=[]
    loss_fn=torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for batch_dl in test_dataloader:
            d,l=batch_dl
            out=lstm(d.to(device))
            bloss=loss_fn(out,l.to(device))
            loss_eval+=bloss.item()
            preds=torch.argmax(out,dim=1)
            accuracy_eval+=accuracy(preds,l.to(device))
            precision_eval+=precision(preds,l.to(device))
            recall_eval+=recall(preds,l.to(device))
            f1_eval+=f1(preds,l.to(device))
            preds_p.extend(preds)
            target_p.extend(l)
    accuracy_eval=accuracy_eval/len(test_dataloader)
    precision_eval=precision_eval/len(test_dataloader)
    recall_eval=recall_eval/len(test_dataloader)
    f1_eval=f1_eval/len(test_dataloader)
    return accuracy_eval,precision_eval,recall_eval,f1_eval,preds_p,target_p

In [48]:
print(eval_func(lstm_5))

(tensor(0.9014, device='cuda:0'), tensor(0.9009, device='cuda:0'), tensor(0.9012, device='cuda:0'), tensor(0.2259, device='cuda:0'))


In [49]:
lstm_7=LSTM1(context7,300,512,4).to(device)

In [50]:
import gc
torch.cuda.empty_cache()
gc.collect()

290

In [51]:
tr_l_7,acc_train_7,precison_train_7,recall_train_7,f1_train_7=train(lstm_7,device)
torch.save(lstm_7,'./lstm7')
print(acc_train_7,precison_train_7,recall_train_7,f1_train_7)

train: 100%|██████████| 59/59 [00:56<00:00,  1.05it/s]


101.71908247470856


train: 100%|██████████| 59/59 [00:57<00:00,  1.02it/s]


97.49471545219421


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


...

8.875397384166718


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


8.398592118173838


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


6.7408052533864975
[tensor(0.2525, device='cuda:0'), tensor(0.2481, device='cuda:0'), tensor(0.2468, device='cuda:0'), tensor(0.2495, device='cuda:0'), tensor(0.2544, device='cuda:0'), tensor(0.2663, device='cuda:0'), tensor(0.4746, device='cuda:0'), tensor(0.7345, device='cuda:0'), tensor(0.8724, device='cuda:0'), tensor(0.9190, device='cuda:0'), tensor(0.9344, device='cuda:0'), tensor(0.9470, device='cuda:0'), tensor(0.9523, device='cuda:0'), tensor(0.9545, device='cuda:0'), tensor(0.9632, device='cuda:0')] [tensor(0.1905, device='cuda:0'), tensor(0.1455, device='cuda:0'), tensor(0.2220, device='cuda:0'), tensor(0.2286, device='cuda:0'), tensor(0.3035, device='cuda:0'), tensor(0.3659, device='cuda:0'), tensor(0.5389, device='cuda:0'), tensor(0.7671, device='cuda:0'), tensor(0.8858, device='cuda:0'), tensor(0.9206, device='cuda:0'), tensor(0.9353, device='cuda:0'), tensor(0.9470, device='cuda:0'), tensor(0.9526, device='cuda:0'), tensor(0.9550, device='cuda:0'), tensor(0.9633, device=

In [53]:
print(eval_func(lstm_7))

(tensor(0.8977, device='cuda:0'), tensor(0.8982, device='cuda:0'), tensor(0.8977, device='cuda:0'), tensor(0.8972, device='cuda:0'))


In [56]:
lstm_9=LSTM1(context9,300,512,4).to(device)
import gc
torch.cuda.empty_cache()
gc.collect()
tr_l_9,acc_train_9,precison_train_9,recall_train_9,f1_train_9=train(lstm_9,device)
torch.save(lstm_9,'./lstm9')
print(acc_train_9,precison_train_9,recall_train_9,f1_train_9)
print(eval_func(lstm_9))

train: 100%|██████████| 59/59 [00:56<00:00,  1.04it/s]


103.29244697093964


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


85.4764791727066


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


...

7.152499441057444


train: 100%|██████████| 59/59 [00:58<00:00,  1.02it/s]


6.1514975018799305


train: 100%|██████████| 59/59 [00:57<00:00,  1.03it/s]


5.340691737830639
[tensor(0.2499, device='cuda:0'), tensor(0.2647, device='cuda:0'), tensor(0.3604, device='cuda:0'), tensor(0.5875, device='cuda:0'), tensor(0.7381, device='cuda:0'), tensor(0.8845, device='cuda:0'), tensor(0.9125, device='cuda:0'), tensor(0.9277, device='cuda:0'), tensor(0.9403, device='cuda:0'), tensor(0.9474, device='cuda:0'), tensor(0.9576, device='cuda:0'), tensor(0.9607, device='cuda:0'), tensor(0.9613, device='cuda:0'), tensor(0.9674, device='cuda:0'), tensor(0.9722, device='cuda:0')] [tensor(0.1916, device='cuda:0'), tensor(0.1873, device='cuda:0'), tensor(0.3467, device='cuda:0'), tensor(0.5907, device='cuda:0'), tensor(0.7598, device='cuda:0'), tensor(0.8907, device='cuda:0'), tensor(0.9176, device='cuda:0'), tensor(0.9311, device='cuda:0'), tensor(0.9411, device='cuda:0'), tensor(0.9485, device='cuda:0'), tensor(0.9579, device='cuda:0'), tensor(0.9611, device='cuda:0'), tensor(0.9622, device='cuda:0'), tensor(0.9676, device='cuda:0'), tensor(0.9723, device='

In [60]:
_,_,_,_,pp5,tp5=eval_func(lstm_5)
def converttonumpy(pp):
    return torch.tensor(pp).detach().numpy()
_,_,_,_,pp7,tp7=eval_func(lstm_7)
_,_,_,_,pp9,tp9=eval_func(lstm_9)
from sklearn.metrics import confusion_matrix
confusion_matrix(converttonumpy(pp5),converttonumpy(tp5))
confusion_matrix(converttonumpy(pp7),converttonumpy(tp7))
confusion_matrix(converttonumpy(pp9),converttonumpy(tp9))

array([[1698,   26,   70,   63],
       [  53, 1835,   11,   12],
       [  93,   23, 1700,  254],
       [  56,   16,  119, 1571]])

In [61]:

confusion_matrix(converttonumpy(pp5),converttonumpy(tp5))

array([[1695,   22,   66,   61],
       [  72, 1856,   28,   46],
       [  78,    9, 1646,  141],
       [  55,   13,  160, 1652]])

In [62]:

confusion_matrix(converttonumpy(pp7),converttonumpy(tp7))

array([[1729,   25,   77,  108],
       [  46, 1847,   17,   23],
       [  86,   17, 1683,  216],
       [  39,   11,  123, 1553]])